In [6]:
import os
import sys
import math
import random
import torch
import numpy as np
import syft as sy
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import ujson as json
import pandas as pd
import re
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [7]:
def loadDataset(path, screenName):

    users = [ f.path for f in os.scandir(path) if f.is_dir() ]
    info = pd.DataFrame(columns= ['accelometer_size', 'timestamp'])

    accelerometer = pd.DataFrame(columns=['x', 'y', 'z', 'screen', 'user', 'magnitude','combine_angle', 'timestamp'])
    
    # Read sensors data from json file and save them in Dataframes
    for i in range(0, len(users)):

        json_files = [pos_json for pos_json in os.listdir(users[i]) if pos_json.endswith('.json')]

        for index, js in enumerate(json_files):
            with open(os.path.join(users[i], js)) as json_file:
                json_text = json.load(json_file)
                accSize = 0                
                js = js.replace('.json','')
                arr = js.split('_')

                for j in json_text['accelerometer']:
                    if screenName in j['screen']:
                        x = j['x']
                        y = j['y']
                        z = j['z']
                        if x == 0 and y == 0:
                            continue
                        screen = j['screen']
                        user = arr[0]
                        m = x**2 + y**2 + z**2
                        m = np.sqrt(m)
                        ca = np.sqrt(y**2 + z**2)
                        timestamp = arr[1]
                        accSize = accSize + 1
                        df = {'x': x, 'y': y, 'z' : z, 'screen' : screen, 'user': user, 'magnitude' : m, 'combine_angle': ca, 'timestamp': timestamp}
                        accelerometer = accelerometer.append(df, ignore_index=True)
                    
                dframe = {'accelometer_size': accSize, 'timestamp': arr[1]}
                info = info.append(dframe, ignore_index=True)

    return accelerometer, info, users

In [8]:
path = 'C:/Users/SouthSystem/Documents/Pessoal/TCC/Impl/sensors_data'
screen = 'MathisisGame'
accelerometer, info, users = loadDataset(path, screen)

In [9]:
path = 'C:/Users/SouthSystem/Documents/Pessoal/TCC/Impl/sensors_data'
screen = 'MemoriaGame'
accelerometerMem, infoMem, usersMem = loadDataset(path, screen)

In [10]:
path = 'C:/Users/SouthSystem/Documents/Pessoal/TCC/Impl/sensors_data'
screen = 'FocusGame'
accelerometerFoc, infoFoc, usersFoc = loadDataset(path, screen)

In [11]:
user_list = list(sorted(set(accelerometer['screen'])))
print(accelerometer['user'].value_counts())

0kjevf3    3851
0irsvx7    2333
0ddm38     2219
00qhiaz    2054
0mmpa11    1803
1uep0cy    1619
0k77as7    1342
1v7m67q    1261
1y7jz3u    1005
0zb4hq6     924
0ku7bb      916
1ii3q4u     864
0z9xv36     819
0zh8j1j     514
1gnhe9f     362
1i8msn6     285
0lnmasb     276
Name: user, dtype: int64


In [12]:
accelerometer['screen'] = '1'
accelerometerMem['screen'] = '2'
accelerometerFoc['screen'] = '3'

In [13]:
p = accelerometer['user'] + "_" + accelerometer['timestamp']
print(p)

0        00qhiaz_1542975613902
1        00qhiaz_1542975613902
2        00qhiaz_1542975613902
3        00qhiaz_1542975613902
4        00qhiaz_1542975613902
                 ...          
22442    1y7jz3u_1538915244185
22443    1y7jz3u_1538915244185
22444    1y7jz3u_1538915244185
22445    1y7jz3u_1538915244185
22446    1y7jz3u_1538915244185
Length: 22447, dtype: object


In [14]:
accelerometer['timestamp'] = accelerometer['user'] + "_" + accelerometer['timestamp'] + "_" + accelerometer['screen']
accelerometerMem['timestamp'] = accelerometerMem['user'] + "_" + accelerometerMem['timestamp'] + "_" + accelerometerMem['screen']
accelerometerFoc['timestamp'] = accelerometerFoc['user'] + "_" + accelerometerFoc['timestamp'] + "_" + accelerometerFoc['screen']

In [15]:
m = pd.merge(accelerometer, accelerometerMem, how = 'outer')
mergedData = pd.merge(m, accelerometerFoc, how = 'outer')
print(mergedData)

              x         y         z screen     user  magnitude  combine_angle  \
0      0.131836  0.796875  0.575195      1  00qhiaz   0.991585       0.982781   
1      0.073242  0.816406  0.584961      1  00qhiaz   1.007007       1.004340   
2      0.088867  0.810547  0.587891      1  00qhiaz   1.005236       1.001300   
3      0.083984  0.807617  0.602539      1  00qhiaz   1.011114       1.007620   
4      0.088867  0.807617  0.587891      1  00qhiaz   1.002875       0.998930   
...         ...       ...       ...    ...      ...        ...            ...   
80072  0.464721  0.840942  0.245361      3  1y7jz3u   0.991641       0.876006   
80073  0.283081  0.889770  0.296142      3  1y7jz3u   0.979554       0.937758   
80074  0.390503  0.827270  0.239502      3  1y7jz3u   0.945637       0.861241   
80075  0.337769  0.817505  0.247314      3  1y7jz3u   0.918458       0.854095   
80076  0.408081  0.850707  0.339111      3  1y7jz3u   1.002611       0.915805   

                     timest

In [16]:
train_dataset = mergedData.sample(frac=0.8, random_state=25)
test_dataset = mergedData.drop(train_dataset.index)

In [17]:
    metrics = ['x','y','z']
    user_list = list(sorted(set(mergedData['user'])))
    datadict = {}
    screen = []
    finaldata = []
    for user in user_list:
        is_user = mergedData['user']==user
        accelerometer_user = mergedData[is_user]    
        filtered = accelerometer_user[['x','y','z','screen','user','timestamp']]        
        time_list = list(sorted(set(filtered['timestamp'])))   
        for idx, metric in enumerate(metrics):
            data = []
            for timestamp in time_list:
                df = filtered[filtered['timestamp']==timestamp]                
                list_df = list(df[metric].head(100))
                if len(list_df) > 99:   
                    #print(metric)                    
                    screen.append(df['screen'].values[0])
                    data.append(list_df)
                    #print(pd.DataFrame(data))
            datadict[idx] = data
        data0 = datadict[0]
        data1 = datadict[1]
        data2 = datadict[2]
        for idx, val in enumerate(data0):
            finaldata.append(val + data1[idx] + data2[idx])

    df = pd.DataFrame(finaldata)  
    data_array = np.asarray(df)
    converted_num = res = list(map(int, screen))
    screen_array = np.asarray(converted_num)
    #print(screen_array)

In [18]:
# Initial project configuration
result = []
project_name = 'Human Activity Recognition'
arch = "Convolution + pooling + convolution + pooling + dense + dense + dense + output"
batch_size = 64
epochs = 50
lr = 0.01
momentum = 0.9
torch.manual_seed(29)

In [19]:
def load(batch_size=64):
    x_train, y_train, x_test, y_test = load_data()
    print(x_train.shape)
    x_train, x_test = x_train.reshape(
        (-1, 3, 1, 100)), x_test.reshape((-1, 3, 1, 100))
    #print(x_train.shape)
    transform = None
    train_set = Data_loader(x_train, y_train, transform)
    test_set = Data_loader(x_test, y_test, transform)
    train_loader = DataLoader(
        train_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader


def load_data():
    X_train, Y_train = format_data(train_dataset)
    X_test, Y_test = format_data(test_dataset)
    return X_train, Y_train, X_test, Y_test

def format_data(dataItem):
    metrics = ['x','y','z']
    user_list = list(sorted(set(dataItem['user'])))
    datadict = {}
    screen = []
    finaldata = []
    for user in user_list:
        is_user = dataItem['user']==user
        accelerometer_user = dataItem[is_user]    
        filtered = accelerometer_user[['x','y','z','screen','user','timestamp']]        
        time_list = list(sorted(set(filtered['timestamp'])))   
        for idx, metric in enumerate(metrics):
            data = []
            for timestamp in time_list:
                df = filtered[filtered['timestamp']==timestamp]                
                list_df = list(df[metric].head(100))
                if len(list_df) > 99:   
                    #print(metric)                    
                    screen.append(df['screen'].values[0])
                    data.append(list_df)
                    #print(pd.DataFrame(data))
            datadict[idx] = data
        data0 = datadict[0]
        data1 = datadict[1]
        data2 = datadict[2]
        for idx, val in enumerate(data0):
            finaldata.append(val + data1[idx] + data2[idx])

    df = pd.DataFrame(finaldata)  
    data_array = np.asarray(df)
    converted_num = res = list(map(int, screen))
    for i in range(len(converted_num)):
        converted_num[i] = converted_num[i] - 1
    #print(converted_num)
    screen_array = np.asarray(converted_num)
    
    X = None
    x_data = data_array
    print(x_data.shape)
    for i in range(len(x_data)):
        print(x_data[i, :])
        row = np.asarray(x_data[i, :])
        row = row.reshape(3, 100).T
        if X is None:
            X = np.zeros((len(x_data), 100, 3))
        X[i] = row
    print(X.shape)
    return X, screen_array

class Data_loader(Dataset):
    def __init__(self, samples, labels, t):
        self.samples = samples
        self.labels = labels
        self.T = t

    def __getitem__(self, index):
        sample, target = self.samples[index], self.labels[index]
        if self.T:
            return self.T(sample), target
        else:
            return sample, target

    def __len__(self):
        return len(self.samples)

In [20]:
# Defining network architecture
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=9, kernel_size=(1, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2), stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=9, out_channels=64, kernel_size=(1, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2), stride=2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=64 * 23, out_features=400),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(in_features=400, out_features=200),
            nn.ReLU()
        )
        self.fc3 = nn.Sequential(
            nn.Linear(in_features=200, out_features=3)
        )

    def forward(self, x):
        #print(len(x))
        out = self.conv1(x)
        #print(out.shape)
        out = self.conv2(out)
        #print(out.shape)
        out = out.reshape(-1, 64 * 23)
        #print(len(out))
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = F.softmax(out, dim=1)
        return out

In [21]:
# Train and plot functions
def train(model, optimizer, train_loader, test_loader):
    n_batch = len(train_loader.dataset) // batch_size
    criterion = nn.CrossEntropyLoss()

    for e in range(epochs):
        model.train()
        correct, total_loss = 0, 0
        total = 0
        for index, (sample, target) in enumerate(train_loader):
            sample, target = sample.to(
                DEVICE).float(), target.to(DEVICE).long()
            sample = sample.view(-1, 3, 1, 100)
            output = model(sample)
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum()

        acc_train = float(correct) * 100.0 / (batch_size * n_batch)
        print(f'Epoch: [{e+1}/{epochs}], loss: {total_loss}, train acc: {acc_train}%')

        # We proceed now to use the test data to evaluate intermediate results without modifying the model (no training)
        model.train(False)
        with torch.no_grad():
            correct, total = 0, 0
            for sample, target in test_loader:
                sample, target = sample.to(
                    DEVICE).float(), target.to(DEVICE).long()
                sample = sample.view(-1, 3, 1, 100)
                output = model(sample)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum()
        acc_test = float(correct) * 100 / total
        print(f'Epoch: [{e+1}/{epochs}], test acc: {float(correct) * 100 / total}%')   
        np.append(result, [acc_train, acc_test])
        result_np = np.array(result, dtype=float)
        np.savetxt('result.csv', result_np, fmt='%.2f', delimiter=',')

In [22]:
# Get GPU if available
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
DEVICE = get_default_device()
DEVICE

device(type='cpu')

In [28]:
train_loader, test_loader = load(
        batch_size=batch_size)

(321, 300)
[ 0.04394531  0.03320312  0.03613281  0.05371094  0.01660156  0.01464844
  0.03320312  0.01660156  0.03125     0.015625    0.02832031  0.06933594
  0.03125     0.07519531  0.06054688 -0.02636719  0.01757812  0.02441406
  0.05859375  0.01757812  0.03417969  0.07128906  0.03222656  0.02929687
  0.04003906  0.04199219  0.16601562  0.03808594  0.02636719  0.06738281
  0.03808594  0.0546875   0.02636719  0.02441406  0.02441406  0.01953125
  0.05273437  0.04785156  0.04199219  0.00976562  0.09179688  0.02636719
  0.0234375   0.03320312  0.02050781  0.04199219  0.0234375   0.00390625
  0.01855469  0.00195312  0.03417969  0.02832031  0.10742188  0.05078125
  0.0078125  -0.0078125   0.03320312  0.03320312  0.04101562  0.03222656
  0.02148438  0.01367188  0.03320312  0.06542969  0.03710938  0.01171875
  0.04394531  0.04101562  0.          0.03320312  0.03515625  0.04589844
  0.02929687  0.0703125  -0.00585938  0.0390625   0.04492188  0.03710938
  0.03222656  0.05566406  0.0234375  -0.

  0.55932617  0.57800293  0.55493164  0.5489502   0.54797363  0.58068848]
[ 0.06689453  0.05566406  0.05810547  0.02685547  0.02197266  0.05297851
  0.07043457  0.03808594  0.07312012  0.05761719  0.05944825  0.06567383
  0.06237793  0.0645752   0.07141113  0.06835938  0.03503418  0.06298828
  0.04748535  0.04199219  0.04614258  0.05285644  0.01904297  0.06726074
  0.03198242  0.03491211  0.02478027  0.06066895  0.03918457  0.06530762
  0.06311035  0.03930664  0.03381348  0.0567627   0.05859375  0.04260254
  0.04797363  0.05517578  0.06298828  0.05957032  0.06347656  0.07043457
  0.03918457  0.0612793   0.04138184 -0.0032959   0.06274414  0.00476074
  0.03991699  0.07080078  0.02209473  0.0625      0.05102539  0.06787109
  0.05383301  0.07214355  0.06726074  0.04321289  0.06347656  0.0369873
  0.05883789  0.04833984  0.04821777  0.06164551  0.01184082  0.03100586
  0.05322266  0.05883789  0.05285644  0.08972168  0.0657959   0.02099609
  0.03723145  0.06518555  0.06115723  0.04382324  0

  5.91675341e-01  5.50660133e-01  5.46753109e-01  5.33080876e-01]
[ 5.73683716e-03  3.50340717e-02 -2.07565515e-03 -1.37936156e-02
  2.72215810e-02  2.13633776e-02 -5.98112354e-03  1.94090884e-02
  2.52688453e-02  7.21438006e-02 -9.88814794e-03  5.73683716e-03
  1.74563546e-02 -5.98112354e-03 -1.57463495e-02  5.06590568e-02
  5.73683716e-03 -4.02838923e-03  4.28465642e-02  3.69883627e-02
  1.35493297e-02  4.28465642e-02  2.33161114e-02  1.74563546e-02
  1.55036198e-02  2.13633776e-02  5.84715493e-02  1.35493297e-02
  9.64386109e-03 -5.98112354e-03  3.78410309e-03  3.11286040e-02
  1.15965959e-02  1.83136901e-03  5.06590568e-02  3.50340717e-02
  2.72215810e-02  2.33161114e-02  7.60508254e-02 -4.02838923e-03
  1.09253526e-01  6.62840456e-02  1.83136901e-03  5.06590568e-02
  5.73683716e-03  3.78410309e-03  3.50340717e-02  1.83136901e-03
  7.69112725e-03  6.62840456e-02  1.15965959e-02  2.72215810e-02
  2.52688453e-02  2.72215810e-02  1.74563546e-02  1.94090884e-02
  1.74563546e-02  4.8706

  1.81030273e-01  1.88354492e-01  1.92260742e-01  2.01049805e-01]
[ 1.69677734e-02  1.84326172e-02  2.52685547e-02  2.67334003e-02
  8.17871094e-03 -6.71386765e-03  9.64355469e-03  5.26123010e-02
  7.20214844e-03  3.21044922e-02  2.31933594e-03  3.50341797e-02
  3.21044922e-02  3.99169922e-02  1.06201172e-02  2.81982422e-02
  1.30615225e-02  4.76074219e-03 -8.54492188e-04  2.52685547e-02
  1.50146494e-02  2.86865234e-02  3.35693359e-02  2.03857422e-02
  2.52685547e-02  5.73730469e-03  1.01318359e-02  1.98974609e-02
  2.96630841e-02  2.67334003e-02  2.91748028e-02  2.91748028e-02
  1.94091797e-02  2.77099609e-02  2.67334003e-02  1.35498056e-02
  2.72216797e-02  8.54492188e-04  2.96630841e-02  2.81982422e-02
  3.06396484e-02  4.43115234e-02  1.35498056e-02 -4.27246094e-03
  2.38037109e-02  2.23388672e-02  6.14013672e-02  5.60302734e-02
  2.72216797e-02  2.96630841e-02 -3.29589820e-03  1.35498056e-02
  2.77099609e-02  5.24902344e-03  2.67334003e-02  1.83105457e-03
  6.22558594e-03  2.2827

  8.65475118e-01  9.09420192e-01  8.97456348e-01  9.92671967e-01]
[ 2.03255508e-02 -8.24567229e-02 -4.21743877e-02 -1.79625094e-01
 -9.46632624e-02 -1.04430042e-01 -4.33942638e-02  3.35092284e-02
  2.69181672e-02 -7.58656636e-02 -7.48885199e-02 -6.63431659e-02
 -7.65969679e-02 -7.12257847e-02 -8.78279135e-02 -7.17143565e-02
 -1.11998245e-01 -5.99963926e-02 -1.20543599e-01 -8.46537501e-02
 -5.48694953e-02 -4.11972441e-02 -9.83259976e-02 -1.38365611e-01
 -7.83069730e-02 -1.30064532e-01 -6.17048405e-02 -7.02501908e-02
 -1.39096901e-01 -9.68618393e-02 -1.20299309e-01 -1.03697181e-01
 -4.87242162e-01 -1.19322166e-01 -5.87749630e-02 -9.19792205e-02
 -8.36781561e-02 -1.28843114e-01 -5.82863912e-02 -7.80626833e-02
 -2.71422267e-01 -6.58561513e-02 -9.14906487e-02 -7.51328096e-02
 -1.10776819e-01 -7.00059086e-02 -3.06991581e-02 -3.37583393e-01
 -3.05846393e-01 -5.07181846e-02 -6.48790076e-02 -5.92635348e-02
 -3.11877318e-02 -3.14320177e-02 -4.31499779e-02 -5.53580672e-02
 -2.14293510e-01 -5.9752

(2, 300)
[ 0.43887329  0.43414307  0.43261719  0.43467712  0.43757629  0.44062805
  0.40751648  0.44625854  0.45417786  0.45904541  0.4556427   0.44311523
  0.45117188  0.46669006  0.47703552  0.47406006  0.49545288  0.48683167
  0.51271057  0.47303772  0.45133972  0.39141846  0.36949158  0.35774231
  0.44456482  0.47068787  0.47090149  0.45294189  0.44737244  0.445755
  0.44273376  0.41859436  0.4145813   0.4118042   0.40814209  0.39978027
  0.39730835  0.40124512  0.41738892  0.35127258  0.39746094  0.41163635
  0.47795105  0.50183105  0.51925659  0.49316406  0.43791199  0.38745117
  0.35317993  0.35244751  0.42591858  0.40982056  0.40760803  0.4090271
  0.40740967  0.42810059  0.40821838  0.39524841  0.35610962  0.35958862
  0.36512756  0.48472595  0.50559998  0.5115509   0.47442627  0.44572449
  0.4357605   0.32723999  0.43389893  0.48382568  0.36125183  0.40643311
  0.4462738   0.42393494  0.39453125  0.40815735  0.41140747  0.38990784
  0.38594055  0.3837738   0.41357422  0.45669

In [213]:
# Load to selected device and traing model
model = Network().to(DEVICE)
optimizer = optim.SGD(params=model.parameters(), lr=lr, momentum=momentum)
train(model, optimizer, train_loader, test_loader)
result = np.array(result, dtype=float)

Epoch: [1/50], loss: 5.4909409284591675, train acc: 40.0%
Epoch: [1/50], test acc: 0.0%
Epoch: [2/50], loss: 5.4704872369766235, train acc: 50.3125%
Epoch: [2/50], test acc: 0.0%
Epoch: [3/50], loss: 5.437556982040405, train acc: 50.625%
Epoch: [3/50], test acc: 0.0%
Epoch: [4/50], loss: 5.402420401573181, train acc: 50.625%
Epoch: [4/50], test acc: 0.0%
Epoch: [5/50], loss: 5.367018938064575, train acc: 50.3125%
Epoch: [5/50], test acc: 0.0%
Epoch: [6/50], loss: 5.3316954374313354, train acc: 50.3125%
Epoch: [6/50], test acc: 0.0%
Epoch: [7/50], loss: 5.293967962265015, train acc: 50.3125%
Epoch: [7/50], test acc: 0.0%
Epoch: [8/50], loss: 5.253796219825745, train acc: 50.625%
Epoch: [8/50], test acc: 0.0%
Epoch: [9/50], loss: 5.218061208724976, train acc: 50.3125%
Epoch: [9/50], test acc: 0.0%
Epoch: [10/50], loss: 5.185092866420746, train acc: 50.3125%
Epoch: [10/50], test acc: 0.0%
Epoch: [11/50], loss: 5.148147106170654, train acc: 50.625%
Epoch: [11/50], test acc: 0.0%
Epoch: [12